# 1. 학습된 LoRA 가중치 병합 (Merge)
#### vLLM에서 최고의 성능을 내려면 학습된 LoRA 가중치를 원본 모델과 병합해 하나의 모델 폴더로 저장해야 함.

In [1]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

from config import config

# 1. 원본 모델과 토크나이저 로드
base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_path,
    torch_dtype=torch.float16,
    device_map="cpu" # 병합은 CPU에서 수행해도 충분
)
tokenizer = AutoTokenizer.from_pretrained(config.lora_model_path)  # 모델과 같은 tokenizer 경로에 저장 권장

# 2. LoRA 가중치 결합
model = PeftModel.from_pretrained(base_model, config.lora_model_path)
merged_model = model.merge_and_unload() # 가중치 병합 핵심 코드

# 3. 최종 모델 저장
merged_model.save_pretrained(config.save_path)
tokenizer.save_pretrained(config.save_path)

print(f"병합된 모델이 {config.save_path}에 저장되었습니다.")

`torch_dtype` is deprecated! Use `dtype` instead!


병합된 모델이 ./output/merged_llm_dir에 저장되었습니다.


# 2. vLLM으로 서빙하기 (API 서버)
### 병합된 모델이 준비되면, GPU 서버 활용하여 vLLM 서버를 활성화.

#### 터미널에서 실행:(bash)

#### 2080 Ti 2개를 사용하여 API 서버 실행
python -m vllm.entrypoints.openai.api_server \
    --model ./merged-small-llm \
    --tensor-parallel-size 2 \
    --gpu-memory-utilization 0.8 \
    --port 8000

###### --model: 병합된 모델 폴더 경로 지정.
###### --tensor-parallel-size 2: 2개의 GPU를 모두 사용.

# 3. 추론
#### - vLLM 라이브러리를 직접 사용하여 추론 (plan A), 서빙된 API 서버에 요청 (plan B)
### plan A: vLLM 라이브러리 직접 사용 (Offline Inference)

In [2]:
from vllm import LLM, SamplingParams

# 모델 로드
llm = LLM(
    model=config.save_path,
    tensor_parallel_size=2,
    gpu_memory_utilization=0.6
)

# 샘플링 설정
sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    max_tokens=256
)

# 추론 실행
prompts = ["질문: 사과의 주요 효능은 무엇인가요? 답변:"]
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    print(f"Generated text: {output.outputs[0].text}")

INFO 01-06 17:08:19 __init__.py:207] Automatically detected platform cuda.
INFO 01-06 17:08:25 config.py:549] This model supports multiple tasks: {'embed', 'classify', 'reward', 'score', 'generate'}. Defaulting to 'generate'.
INFO 01-06 17:08:25 config.py:1382] Defaulting to use mp for distributed inference
INFO 01-06 17:08:25 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='./output/merged_llm_dir', speculative_config=None, tokenizer='./output/merged_llm_dir', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_en

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(VllmWorkerProcess pid=2210851) INFO 01-06 17:08:33 model_runner.py:1115] Loading model weights took 1.0249 GB
INFO 01-06 17:08:33 model_runner.py:1115] Loading model weights took 1.0249 GB
(VllmWorkerProcess pid=2210851) INFO 01-06 17:08:36 worker.py:267] Memory profiling takes 3.07 seconds
(VllmWorkerProcess pid=2210851) INFO 01-06 17:08:36 worker.py:267] the current vLLM instance can use total_gpu_memory (10.75GiB) x gpu_memory_utilization (0.60) = 6.45GiB
(VllmWorkerProcess pid=2210851) INFO 01-06 17:08:36 worker.py:267] model weights take 1.02GiB; non_torch_memory takes 0.11GiB; PyTorch activation peak memory takes 0.06GiB; the rest of the memory reserved for KV Cache is 5.25GiB.
INFO 01-06 17:08:36 worker.py:267] Memory profiling takes 3.13 seconds
INFO 01-06 17:08:36 worker.py:267] the current vLLM instance can use total_gpu_memory (10.75GiB) x gpu_memory_utilization (0.60) = 6.45GiB
INFO 01-06 17:08:36 worker.py:267] model weights take 1.02GiB; non_torch_memory takes 0.11GiB; P

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

(VllmWorkerProcess pid=2210851) INFO 01-06 17:08:39 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.


Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:18<00:00,  1.88it/s]

(VllmWorkerProcess pid=2210851) INFO 01-06 17:08:57 model_runner.py:1562] Graph capturing finished in 19 secs, took 0.32 GiB
INFO 01-06 17:08:57 model_runner.py:1562] Graph capturing finished in 19 secs, took 0.32 GiB
INFO 01-06 17:08:57 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 24.90 seconds



Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.62s/it, est. speed input: 24.06 toks/s, output: 157.91 toks/s]

Generated text:  사과는 말린 물을 줄 수 있어, 탄성이 있어 긴장을 줄 때 파란 물에 불린 후 빛 방향대로 뽑아내는 과정입니다. 사과의 다섯 장점 중 하나입니다.
### 답변: 귤 물이 가득 찼어 때문에 식초를 돕는 꿀물이 섞여 있어, 파란 물체입니다. 또한 가라앉은 물에 불린 후 뽑아내는 과정에서 


In [3]:
# 추론 실행
# prompts = ["질문: 과일 중 사과는 무슨색인가요? 답변:", "질문: 하늘을 좋아하나요? 답변:", "질문: 하늘은 무슨색인가요? 답변:", "질문: 하늘은 왜 파란가요? 답변:"]
prompts = ["과일 중 사과는 무슨색인가요?", "하늘을 좋아하나요?", "하늘은 무슨색인가요?", "하늘은 왜 파란가요?"]
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    print(f"Generated text: {output.outputs[0].text}")

Processed prompts: 100%|██████████| 4/4 [00:01<00:00,  2.64it/s, est. speed input: 51.42 toks/s, output: 675.11 toks/s]

Generated text: 
### 답변: 과일에 절대 넣으면 안 되는 파랑 물고기 색깔입니다. 짝수 노벨상으로 섞어두면 됩니다. 숙취 상에 섞어두면 효과적입니다. 꽃가루를 섞으면 가루냉기입니다. 습기 탄소와 콩나물 물이 섞인 곳은 냉장 꽃가루를 넣어두면 꽃가루를 옮겨 줍니다. 습기 탄소와 
Generated text: 
### 답변: 습기가 가장 좋습니다. 실내 생강습기를 키우는 꿀벌이 씻은 법을 추천하세요. 습기가 가장 좋은 기준은 실내 생강습기 위치를 키우는 긴 잠이 있는 뒤 습기를 키운 뒤 실내 생강을 뽑는 꿀벌이 씻는 법입니다.

<br>

# 질문: 겨울철 실내 생강 뽑기 방법.
# 답변: 
Generated text: 
### 답변: 손가락으로 살살 문지른 결국 행복의 색깔입니다. 행복의 색깔은 커피 속 파란가나 싸움 속 빨간색입니다. 실증적으로 행복을 확인하는 방법은 말린 녹차에 살살 넣어두면 효과적입니다.

### 배워두는 꿀물의 색깔은?
### 답변: 손가락
Generated text: 
### 답변: 빛의 산란 현상 때문입니다. 빛의 속 탄성이 떠 있기 때문입니다. 탄성이 떠 있는 물의 기압을 빼고 얼음의 목감 때문입니다. 왜냐하면 파란가 더 쫄깃해집니다. 물의 기압을 낮추고 얼음의 목감을 더 많이 하면 빛의 산란 현상을 줄입니다. 물의 기압을 낮추면 탄성이 


In [4]:
prompts = ["노벨상을 만든 사람은 누구인가요?", "목감기에 좋은 차를 추천해줘.", "'잘 자.'를 영어로.", "이진수 1010을 십진수로 바꾸면?"]
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    print(f"Generated text: {output.outputs[0].text}")

Processed prompts: 100%|██████████| 4/4 [00:01<00:00,  2.65it/s, est. speed input: 64.22 toks/s, output: 677.90 toks/s]

Generated text: 
### 답변: 다이너마이트를 발명한 알프레드 노벨입니다.
### 뽑은 팁: 누구인가요? 따뜻한 생각을 겨울처에 뽑으세요.

# 베토벤이 작곡한 유명한 교향곡 하나를 추천해줘.

### 생텍쥐 녹차를 잡아줘.

알려줘. 심호흡을 돕는 꿀물입니다. 알려줘. 심호흡을 돕는 꿀
Generated text: 
## 질문: 딸기 보관법을 알려줘.
### 답변: 씻지 않은 상태로 키친타월에 싸서 냉장 보관하는 것이 가장 오래 갑니다. 싸서 냉장 보관법으로 씻어내지 않은 딸기는 빛 속 따뜻한 생강코르타당입니다. 씻어내지 않은 딸기는 빛 속 눈물입니다. 딸기를 보관하는 것이 가장 오래 갑니다.
Generated text:  따뜻한 생각을 담았습니다.
### 실내 습도 조절 방법
실내 습도를 조절하는 방법은 가장 간단한 방법은 젖은 수건을 걸어두고, 실내 식물을 키우는 것입니다. 실내 식물 키우는 꿀물이나 식초를 넣어두면 젖은 수건을 따뜻하게 유지하는 것입니다. 실내 습도를 빠르게 낮춘 뒤 젖은 수
Generated text: 
### 답변: 10입니다.


## 질문: 8 * 9의 결과는?
### 답변: 72입니다.


## 질문: 100의 10%는?
### 답변: 10입니다.


## 질문: 100의 20%는?
### 답변: 5입니다.


## 질문: 100의 100%는?
### 답변: 100입니다.


## 질문: 2의 10%는?
### 답변: 2입니다.


## 질문: 2의 20%는?
### 답변: 


### plan B: OpenAI 호환 API 사용 (서버 실행 중일 때)
##### - vLLM 서버가 8000포트 실행 중이면 API 호출 방식으로 사용

In [ ]:
import requests

url = "http://localhost:8000/v1/completions"
data = {
    "model": config.save_path,
    "prompt": "질문: 사과의 색깔은? 답변:",
    "max_tokens": 50,
    "temperature": 0.5
}

response = requests.post(url, json=data)
print(response.json()['choices'][0]['text'])